In [3]:
import yfinance as yf
import pandas as pd
# Definir los símbolos de los índices
BBVA_ticker = "BBVA.MC" # S&P 500
BANCO_SANTANDER_ticker = "SAN.MC" # IBEX 35
# Definir el rango de fechas
start_date = "2000-01-01"
end_date = "2024-01-01"
# Descargar los datos históricos desde Yahoo Finance
BBVA_data = yf.download(BBVA_ticker, start=start_date, end=end_date)
SANTANDER_data = yf.download(BANCO_SANTANDER_ticker, start=start_date, end=end_date)

C:\Users\aserr\AppData\Local\Temp\ipykernel_27164\2310877257.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  BBVA_data = yf.download(BBVA_ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\aserr\AppData\Local\Temp\ipykernel_27164\2310877257.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  SANTANDER_data = yf.download(BANCO_SANTANDER_ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [4]:
# Mostrar las primeras filas de cada dataset
print("S&P 500 Data:")
print(BBVA_data.head())
print("\nIBEX 35 Data:")
print(SANTANDER_data.head())
# Guardar los datos en CSV (opcional)
BBVA_data.to_csv("bbva.csv")
SANTANDER_data.to_csv("satander.csv")

S&P 500 Data:
Price          Close      High       Low      Open    Volume
Ticker       BBVA.MC   BBVA.MC   BBVA.MC   BBVA.MC   BBVA.MC
Date                                                        
2000-01-03  4.115112  4.155741  4.106406  4.135427   8244257
2000-01-04  4.007736  4.088993  3.993226  4.062875   8522096
2000-01-05  3.917772  3.990323  3.900359  3.970008  12159826
2000-01-06  3.917772  3.917772  3.917772  3.917772         0
2000-01-07  3.967106  4.001931  3.926478  4.001931  62261944

IBEX 35 Data:
Price          Close      High       Low      Open   Volume
Ticker        SAN.MC    SAN.MC    SAN.MC    SAN.MC   SAN.MC
Date                                                       
2000-01-03  2.659118  2.696505  2.628742  2.687158  8797337
2000-01-04  2.596029  2.640425  2.572662  2.626405  8811013
2000-01-05  2.532940  2.579673  2.511910  2.546960  9333517
2000-01-06  2.532940  2.532940  2.532940  2.532940        0
2000-01-07  2.645100  2.652110  2.549297  2.551634  9603132
